# Setup

In [ ]:
import os
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import butter, filtfilt, find_peaks

# Set Data Directory

In [ ]:
data_dir = Path(__file__).parent / 'sample_data' if '__file__' in dir() else Path('sample_data')
data_dir = data_dir.resolve()

print(f'Data directory: {data_dir}')
for file in Path(data_dir).rglob('*.txt'):
    if file.is_file():
        print(file)

# Load Signals

In [ ]:
def load_txt_file(file_path):
    """
    Load spectral data from a given txt file, automatically detecting headers or metadata.

    Args:
        file_path (str or Path): Path to the text file.

    Returns:
        pd.DataFrame: DataFrame with two columns ['Wavelength', 'Intensity'].
    """
    data_started = False
    wavelengths = []
    intensities = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip empty lines
            if not line:
                continue

            # Detect the spectral data start (for non-best-fit files)
            if line.startswith('>>>>>Begin Spectral Data<<<<<'):
                data_started = True
                continue

            # Check if line contains two numerical columns
            parts = line.split()
            if len(parts) == 2:
                try:
                    wavelength, intensity = float(parts[0]), float(parts[1])
                    wavelengths.append(wavelength)
                    intensities.append(intensity)
                    data_started = True  # for BestFit files, data immediately starts
                except ValueError:
                    # Skip lines that don't have numerical data
                    if data_started:
                        # If data previously started but now hit non-numerical, break
                        break
                    else:
                        continue

    return pd.DataFrame({"Wavelength": wavelengths, "Intensity": intensities})


In [ ]:
def load_patient_signals(data_dir, patient):
    """
    Loads the spectra and best fit signals for a given patient.

    Args:
        data_dir (str): The base directory containing the patient data.
        patient (tuple): A tuple containing the fit type ('good_fit' or 'bad_fit') and patient number (str).

    Returns:
        tuple: A tuple containing the spectra DataFrame and the best fit DataFrame.
    """
    patient_dir = os.path.join(data_dir, patient[0], patient[1])

    best_fit_path = None
    for file in Path(patient_dir).rglob('*BestFit.txt'):
        if file.is_file():
            best_fit_path = file
            break

    if best_fit_path is None:
        raise FileNotFoundError(f"No BestFit file found for patient {patient} in {patient_dir}")

    spectra_path = best_fit_path.with_name(best_fit_path.name.replace('_BestFit', ''))
    if not spectra_path.is_file():
         raise FileNotFoundError(f"Spectra file not found for patient {patient} at {spectra_path}")


    spectra_df = load_txt_file(spectra_path)
    best_fit_df = load_txt_file(best_fit_path)

    return spectra_df, best_fit_df

# Detrend

In [ ]:
def detrend_signal(df, cutoff_frequency, filter_order=3):
    """
    Detrends a signal using a high-pass Butterworth filter.

    Parameters:
    df (DataFrame): Input DataFrame with 'Wavelength' and 'Intensity' columns.
    cutoff_frequency (float): Cutoff frequency for the high-pass filter (Hz).
    filter_order (int): Order of the Butterworth filter (default=3).

    Returns:
    DataFrame: DataFrame with an additional 'Detrended_Intensity' column.
    """
    # Ensure the DataFrame is sorted by Wavelength
    df = df.sort_values(by="Wavelength").reset_index(drop=True)

    # Sampling frequency calculation from Wavelength spacing
    sampling_interval = df['Wavelength'].diff().mean()
    sampling_frequency = 1 / sampling_interval

    # Normalize cutoff frequency
    nyquist_freq = 0.5 * sampling_frequency
    normal_cutoff = cutoff_frequency / nyquist_freq

    # High-pass Butterworth filter design
    b, a = butter(filter_order, normal_cutoff, btype='high', analog=False)

    # Apply filter using filtfilt for zero-phase filtering
    detrended_intensity = filtfilt(b, a, df['Intensity'])

    # Add detrended data to DataFrame
    df['Detrended'] = detrended_intensity

    return df


# Peak Detection

In [ ]:
def detect_peaks(df, column, prominence=0.005):
    peaks_indices, _ = find_peaks(df[column], prominence=prominence)
    peaks_df = df.iloc[peaks_indices].reset_index(drop=True)
    return peaks_df

def detect_valleys(df, column, prominence=0.005):
    valleys_indices, _ = find_peaks(-df[column], prominence=prominence)
    valleys_df = df.iloc[valleys_indices].reset_index(drop=True)
    return valleys_df

# LTA Peak Detection Replication

Replicating the peak detection algorithm from LTA software based on config parameters:
- **CRITICAL: Signal must be detrended first** (high-pass Butterworth filter removes baseline)
- Detection ranges: Normal (760-1070nm), Small (750-940nm)
- Moving average smoothing: 9-27nm applied twice for peak detection
- Broader 22-67nm smoothing used only for amplitude RMS calculation
- Spacing formula: `(λ_cur - λ_prev) > constant + λ_prev / denominator`
- Nominal thickness from peak count: `thickness = -628 + 485 * n_peaks` (Normal)

**Why detrending is essential:** Raw spectral data has a strong monotonic baseline that obscures the interference oscillations. The detrending step (same as `Visualize_Data.ipynb`) removes this baseline and reveals the peaks.


In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import Literal

@dataclass
class LTAPeakDetectionConfig:
    """Configuration for LTA-style peak detection."""
    # Detection type and ranges
    detection_type: Literal['Normal', 'Small'] = 'Normal'
    range_normal: tuple[float, float] = (760, 1070)
    range_small: tuple[float, float] = (750, 940)
    range_offset: float = 10  # ±offset expands the detection window
    
    # Smoothing parameters
    moving_avg_width_peaks_nm: tuple[float, float] = (9, 27)  # min, max for peak detection
    moving_avg_width_rms_nm: tuple[float, float] = (22.6778, 67.144)  # for spectrum RMS
    num_moving_avg_passes: int = 2
    
    # Amplitude RMS
    rms_wavelength_range: tuple[float, float] = (750, 920)
    amplitude_rms_threshold: float = 0.0012
    
    # Peak spacing formula: (λ_cur - λ_prev) > constant + λ_prev / denominator
    spacing_coeffs_same: tuple[float, float] = (11, 140)  # constant, denominator for same-type peaks
    spacing_coeffs_interleaved: tuple[float, float] = (7, 70)  # for interleaved peaks
    
    # Intensity separation
    min_intensity_gap: float = 0.00001
    
    # Goodness of Peaks (GOP)
    gop_threshold: float = 75
    gop_trim_percent: float = 20  # drop top N% of inverse wavelength diffs
    
    # Nominal thickness coefficients: thickness = constant + slope * n_peaks
    thickness_coeffs_normal: tuple[float, float] = (-628, 485)
    thickness_coeffs_small: tuple[float, float] = (-866, 780)


def boxcar_smooth(intensity: np.ndarray, width: int) -> np.ndarray:
    """Apply boxcar (uniform) moving average smoothing."""
    if width < 1:
        return intensity
    kernel = np.ones(width) / width
    # Use 'same' mode and handle edges
    return np.convolve(intensity, kernel, mode='same')


def moving_average_nm(wavelength: np.ndarray, intensity: np.ndarray, width_nm: float) -> np.ndarray:
    """Apply moving average with width specified in nm."""
    avg_spacing = np.mean(np.diff(wavelength))
    width_samples = max(1, int(round(width_nm / avg_spacing)))
    return boxcar_smooth(intensity, width_samples)


def compute_amplitude_rms(wavelength: np.ndarray, intensity: np.ndarray, 
                          wl_range: tuple[float, float]) -> float:
    """Compute amplitude RMS over specified wavelength range."""
    mask = (wavelength >= wl_range[0]) & (wavelength <= wl_range[1])
    if not np.any(mask):
        return 0.0
    return np.sqrt(np.mean(intensity[mask] ** 2))


def find_local_maxima(intensity: np.ndarray) -> np.ndarray:
    """Find indices of local maxima in intensity array."""
    maxima = []
    for i in range(1, len(intensity) - 1):
        if intensity[i] > intensity[i-1] and intensity[i] > intensity[i+1]:
            maxima.append(i)
    return np.array(maxima, dtype=int)


def check_spacing(wavelength_current: float, wavelength_previous: float,
                  constant: float, denominator: float) -> bool:
    """Check if peak spacing satisfies: (λ_cur - λ_prev) > constant + λ_prev / denominator."""
    min_distance = constant + wavelength_previous / denominator
    return (wavelength_current - wavelength_previous) > min_distance


def compute_gop(wavelengths: np.ndarray, trim_percent: float) -> float:
    """
    Compute Goodness of Peaks (GOP).
    Lower GOP = better uniformity of peak spacing.
    """
    if len(wavelengths) < 2:
        return float('inf')
    
    # Compute inverse wavelength differences
    inv_wl_diffs = 1.0 / np.diff(wavelengths)
    
    # Trim top N% (largest values = smallest spacing = worst uniformity)
    n_trim = max(0, int(len(inv_wl_diffs) * trim_percent / 100))
    if n_trim > 0 and n_trim < len(inv_wl_diffs):
        sorted_diffs = np.sort(inv_wl_diffs)
        trimmed = sorted_diffs[:-n_trim]
    else:
        trimmed = inv_wl_diffs
    
    # Return RMS of trimmed differences
    return np.sqrt(np.mean(trimmed ** 2)) if len(trimmed) > 0 else float('inf')


def lta_peak_detection(wavelength: np.ndarray, intensity: np.ndarray,
                       config: LTAPeakDetectionConfig = None) -> dict:
    """
    Replicate LTA software peak detection algorithm.
    
    Algorithm steps (from peak_detection_findings.md):
    1. Preprocess with boxcar 11 twice
    2. Compute amplitude RMS using BROAD smoothing (for quality gate only)
    3. Smooth preprocessed signal with NARROW kernel for peak detection
    4. Mask to detection window (Normal or Small range ± offset)
    5. Find local maxima above amplitude threshold
    6. Enforce spacing formula and intensity gap
    7. Compute GOP
    8. Derive nominal thickness from peak count
    
    Key insight: Broad smoothing (22-67nm) is ONLY for RMS calculation,
    narrow smoothing (9-27nm twice) is for peak detection.
    """
    config = config or LTAPeakDetectionConfig()
    
    # Step 1: Preprocessing (boxcar width 11 applied twice)
    preprocessed = boxcar_smooth(intensity, 11)
    preprocessed = boxcar_smooth(preprocessed, 11)
    
    # Step 2: Compute amplitude RMS using BROAD smoothing (22-67nm) - for quality gate only
    avg_width_rms = np.mean(config.moving_avg_width_rms_nm)
    rms_smoothed = moving_average_nm(wavelength, preprocessed, avg_width_rms)
    amplitude_rms = compute_amplitude_rms(wavelength, rms_smoothed, config.rms_wavelength_range)
    passed_rms_gate = amplitude_rms > config.amplitude_rms_threshold
    
    # Step 3: For peak detection, apply ONLY the NARROW smoothing (9-27nm) twice
    # Critical: Do NOT apply broad smoothing here - it destroys the oscillations
    smoothed_for_peaks = preprocessed.copy()
    avg_width_peaks = np.mean(config.moving_avg_width_peaks_nm)
    for _ in range(config.num_moving_avg_passes):
        smoothed_for_peaks = moving_average_nm(wavelength, smoothed_for_peaks, avg_width_peaks)
    
    # Step 4: Select detection window based on type and apply offset
    wl_min, wl_max = config.range_normal if config.detection_type == 'Normal' else config.range_small
    wl_min -= config.range_offset
    wl_max += config.range_offset
    
    # Mask to detection window
    window_mask = (wavelength >= wl_min) & (wavelength <= wl_max)
    wl_window = wavelength[window_mask]
    intensity_window = smoothed_for_peaks[window_mask]
    
    # Step 5: Find local maxima AND filter by amplitude threshold
    local_max_indices = find_local_maxima(intensity_window)
    
    # For detrended data (oscillates around zero), use absolute value for threshold check
    # This ensures peaks are significant regardless of whether they're above or below zero
    if len(local_max_indices) > 0:
        peak_intensities_raw = intensity_window[local_max_indices]
        # Use absolute value - peaks should have significant amplitude regardless of sign
        above_threshold_mask = np.abs(peak_intensities_raw) > config.amplitude_rms_threshold
        local_max_indices = local_max_indices[above_threshold_mask]
    
    if len(local_max_indices) == 0:
        return {
            'peak_wavelengths': np.array([]),
            'peak_intensities': np.array([]),
            'peak_indices': np.array([], dtype=int),
            'amplitude_rms': amplitude_rms,
            'gop': float('inf'),
            'nominal_thickness': 0,
            'n_peaks': 0,
            'passed_quality': False,
            'passed_rms_gate': passed_rms_gate,
            'passed_gop_gate': False
        }
    
    # Step 6: Enforce spacing formula and intensity gap sequentially
    accepted_peaks = []
    constant_same, denom_same = config.spacing_coeffs_same
    
    for idx in local_max_indices:
        wl_current = wl_window[idx]
        int_current = intensity_window[idx]
        
        if len(accepted_peaks) == 0:
            accepted_peaks.append(idx)
        else:
            last_idx = accepted_peaks[-1]
            wl_prev = wl_window[last_idx]
            int_prev = intensity_window[last_idx]
            
            # Check spacing formula: (λ_cur - λ_prev) > constant + λ_prev / denominator
            if check_spacing(wl_current, wl_prev, constant_same, denom_same):
                # Check intensity gap: |I_cur - I_prev| > min_intensity_gap
                if abs(int_current - int_prev) > config.min_intensity_gap:
                    accepted_peaks.append(idx)
    
    accepted_peaks = np.array(accepted_peaks)
    peak_wavelengths = wl_window[accepted_peaks]
    peak_intensities = intensity_window[accepted_peaks]
    
    # Map back to original indices
    original_indices = np.where(window_mask)[0][accepted_peaks]
    
    # Step 7: Compute GOP
    gop = compute_gop(peak_wavelengths, config.gop_trim_percent)
    passed_gop_gate = gop <= config.gop_threshold
    
    # Step 8: Derive nominal thickness from peak count
    n_peaks = len(accepted_peaks)
    if config.detection_type == 'Normal':
        constant, slope = config.thickness_coeffs_normal
    else:
        constant, slope = config.thickness_coeffs_small
    nominal_thickness = constant + slope * n_peaks
    
    passed_quality = passed_rms_gate and passed_gop_gate
    
    return {
        'peak_wavelengths': peak_wavelengths,
        'peak_intensities': peak_intensities,
        'peak_indices': original_indices,
        'amplitude_rms': amplitude_rms,
        'gop': gop,
        'nominal_thickness': nominal_thickness,
        'n_peaks': n_peaks,
        'passed_quality': passed_quality,
        'passed_rms_gate': passed_rms_gate,
        'passed_gop_gate': passed_gop_gate
    }


print('✅ LTA peak detection functions defined')


In [ ]:
# Demo: Apply LTA peak detection to a sample
# KEY FIX: Detrend the signal first to remove baseline and reveal oscillations!
patient = ('good_fit', '1')
spectra_df, best_fit_df = load_patient_signals(data_dir, patient)

# CRITICAL: Detrend the signal first (same as Visualize_Data.ipynb)
# This removes the monotonic baseline and reveals the interference oscillations
cutoff_frequency = 0.008  # Same as Visualize_Data.ipynb
spectra_df = detrend_signal(spectra_df, cutoff_frequency)

# Run LTA detection on DETRENDED spectra
config = LTAPeakDetectionConfig(detection_type='Normal')
wavelength = spectra_df['Wavelength'].values
intensity = spectra_df['Detrended'].values  # Use detrended, not raw Intensity!

result = lta_peak_detection(wavelength, intensity, config)

# Format GOP (handle infinity case)
gop_value = result['gop']
gop_display = f'{gop_value:.2f}' if gop_value != float('inf') else 'inf (no valid peaks)'

print(f'📊 LTA Peak Detection Results for {patient}')
print(f'   Peaks found: {result["n_peaks"]}')
print(f'   Peak wavelengths: {result["peak_wavelengths"]}')
print(f'   Amplitude RMS: {result["amplitude_rms"]:.6f} (threshold: {config.amplitude_rms_threshold})')
print(f'   GOP: {gop_display} (threshold: {config.gop_threshold})')
print(f'   Nominal thickness: {result["nominal_thickness"]:.1f} nm')
print(f'   ✅ Passed RMS gate: {result["passed_rms_gate"]}')
print(f'   ✅ Passed GOP gate: {result["passed_gop_gate"]}')
print(f'   ✅ Overall quality: {"PASS" if result["passed_quality"] else "FAIL"}')


In [ ]:
# Debug: Visualize why detrending is critical for peak detection
import numpy as np

patient = ('good_fit', '1')
spectra_df_raw, _ = load_patient_signals(data_dir, patient)

# Apply detrending (same parameters as Visualize_Data.ipynb)
cutoff_frequency = 0.008
spectra_df_detrended = detrend_signal(spectra_df_raw.copy(), cutoff_frequency)

wavelength = spectra_df_detrended['Wavelength'].values
raw_intensity = spectra_df_raw['Intensity'].values
detrended_intensity = spectra_df_detrended['Detrended'].values

config = LTAPeakDetectionConfig()

# Apply boxcar preprocessing to detrended signal
preprocessed = boxcar_smooth(detrended_intensity, 11)
preprocessed = boxcar_smooth(preprocessed, 11)

# Apply narrow smoothing for peak detection
avg_width_peaks = np.mean(config.moving_avg_width_peaks_nm)
smoothed_for_peaks = preprocessed.copy()
for _ in range(config.num_moving_avg_passes):
    smoothed_for_peaks = moving_average_nm(wavelength, smoothed_for_peaks, avg_width_peaks)

# Plot comparison: raw vs detrended
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=['Raw Intensity (monotonic baseline - no visible peaks)', 
                                   'Detrended Intensity (oscillations revealed - peaks visible!)'])

# Row 1: Raw intensity
fig.add_trace(go.Scatter(x=wavelength, y=raw_intensity, name='Raw', line=dict(color='gray')), row=1, col=1)

# Row 2: Detrended + smoothed
fig.add_trace(go.Scatter(x=wavelength, y=detrended_intensity, name='Detrended', line=dict(color='blue', width=1)), row=2, col=1)
fig.add_trace(go.Scatter(x=wavelength, y=smoothed_for_peaks, name='Smoothed for peaks', line=dict(color='green', width=2)), row=2, col=1)

# Add detection window
for row in [1, 2]:
    fig.add_vrect(x0=750, x1=1080, fillcolor='yellow', opacity=0.1, layer='below', line_width=0, row=row, col=1)

fig.update_layout(
    title='Why Detrending is Critical: Raw vs Detrended Signal',
    height=600, width=1000, showlegend=True
)
fig.show()

# Count peaks on detrended signal
window_mask = (wavelength >= 750) & (wavelength <= 1080)
peaks_detrended = find_local_maxima(smoothed_for_peaks[window_mask])
peaks_raw = find_local_maxima(boxcar_smooth(boxcar_smooth(raw_intensity, 11), 11)[window_mask])
print(f'❌ Raw signal: {len(peaks_raw)} local maxima found (baseline dominates)')
print(f'✅ Detrended signal: {len(peaks_detrended)} local maxima found (oscillations visible!)')


In [ ]:
# Visualize: Compare LTA vs scipy peak detection on DETRENDED signal
fig = go.Figure()

# Plot detrended spectrum (from Cell 13 - spectra_df has 'Detrended' column)
fig.add_trace(go.Scatter(
    x=wavelength, y=intensity,  # intensity is spectra_df['Detrended'].values from Cell 13
    mode='lines', name='Detrended Spectrum',
    line=dict(color='blue', width=1)
))

# Plot LTA detected peaks (only if peaks were found)
n_peaks = result['n_peaks']
if n_peaks > 0:
    fig.add_trace(go.Scatter(
        x=result['peak_wavelengths'],
        y=intensity[result['peak_indices']],
        mode='markers', name=f'LTA Peaks (n={n_peaks})',
        marker=dict(size=12, symbol='triangle-up', color='red')
    ))

# Also show scipy peaks for comparison (on detrended signal!)
scipy_peaks = detect_peaks(spectra_df, 'Detrended', prominence=0.0001)
fig.add_trace(go.Scatter(
    x=scipy_peaks['Wavelength'],
    y=scipy_peaks['Detrended'],
    mode='markers', name=f'Scipy Peaks (n={len(scipy_peaks)})',
    marker=dict(size=8, symbol='circle', color='green', opacity=0.7)
))

# Add detection window shading
detection_range = config.range_normal
fig.add_vrect(
    x0=detection_range[0] - config.range_offset,
    x1=detection_range[1] + config.range_offset,
    fillcolor='yellow', opacity=0.1,
    layer='below', line_width=0,
    annotation_text='Detection Window', annotation_position='top left'
)

# Format GOP display (handle infinity)
gop_display = f'{result["gop"]:.2f}' if result['gop'] != float('inf') else 'inf'

fig.update_layout(
    title=f'LTA Peak Detection Comparison (Detrended) - {patient}<br>'
          f'<sup>LTA Peaks: {n_peaks} | GOP: {gop_display} | Nominal Thickness: {result["nominal_thickness"]:.0f} nm</sup>',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Detrended Intensity',
    height=500, width=900,
    hovermode='x unified',
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01)
)

fig.show()


In [ ]:
# Compare LTA detection across good_fit vs bad_fit samples (with detrending!)
results_summary = []
cutoff_frequency = 0.008  # Same as used in demo

for fit_type in ['good_fit', 'bad_fit']:
    for i in range(1, 11):
        patient = (fit_type, str(i))
        try:
            spectra_df_sample, _ = load_patient_signals(data_dir, patient)
            
            # CRITICAL: Detrend first!
            spectra_df_sample = detrend_signal(spectra_df_sample, cutoff_frequency)
            
            wl = spectra_df_sample['Wavelength'].values
            intens = spectra_df_sample['Detrended'].values  # Use detrended!
            
            res = lta_peak_detection(wl, intens, config)
            
            # Handle infinity GOP for display
            gop_value = res.get('gop', float('inf'))
            gop_display = gop_value if gop_value != float('inf') else np.nan
            
            results_summary.append({
                'fit_type': fit_type,
                'sample': i,
                'n_peaks': res.get('n_peaks', 0),
                'amplitude_rms': res.get('amplitude_rms', 0),
                'gop': gop_display,
                'nominal_thickness': res.get('nominal_thickness', 0),
                'passed_quality': res.get('passed_quality', False)
            })
        except Exception as e:
            print(f'⚠️ Error processing {patient}: {e}')

summary_df = pd.DataFrame(results_summary)
print('📊 LTA Peak Detection Summary (with detrending):')
print(summary_df.to_string(index=False))

# Show average metrics by fit type
print('\n📈 Average metrics by fit type:')
print(summary_df.groupby('fit_type')[['gop', 'n_peaks', 'amplitude_rms']].mean().round(4))


# Plot Signals

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define dropdown options
patient_dropdown = widgets.Dropdown(
    options = [f'{fit}_{i}_{col}' for fit in ['good_fit', 'bad_fit'] for i in range(1, 11) for col in ['Intensity', 'Detrended']],
    value = 'good_fit_1_Detrended',
    description = 'Patient:',
)

plot_output = widgets.Output()

display(patient_dropdown, plot_output)

In [ ]:
def render_plot(patient_id):

    unpacked = tuple(patient_id.rsplit('_', 2))
    patient = unpacked[0:2]
    y_column = unpacked[2]
    x_column = 'Wavelength'

    spectra_df, best_fit_df = load_patient_signals(data_dir, patient)

    cutoff_frequency = 0.008
    filter_order = 3 # default of 3
    peak_prominence = 0.0001

    spectra_df = detrend_signal(spectra_df, cutoff_frequency, filter_order)
    best_fit_df = detrend_signal(best_fit_df, cutoff_frequency, filter_order)

    spectra_peaks = detect_peaks(spectra_df, y_column, peak_prominence)
    best_fit_peaks = detect_peaks(best_fit_df, y_column, peak_prominence)

    spectra_valleys = detect_valleys(spectra_df, y_column, peak_prominence)
    best_fit_valleys = detect_valleys(best_fit_df, y_column, peak_prominence)

    with plot_output:
        plot_output.clear_output(wait=True)

        print(f"Rendering plot for patient: {patient}")  # Optional debug print

        fig = go.Figure()

        # Plot Spectra data
        fig.add_trace(go.Scatter(
            x=spectra_df[x_column],
            y=spectra_df[y_column],
            mode='lines',
            name='Spectra',
            line=dict(color='blue')
        ))

        # Plot BestFit data
        fig.add_trace(go.Scatter(
            x=best_fit_df[x_column],
            y=best_fit_df[y_column],
            mode='lines',
            name='Best Fit',
            line=dict(dash='dot', color='red')
        ))

        # Plot detected peaks
        fig.add_trace(go.Scatter(
            x=spectra_peaks[x_column],
            y=spectra_peaks[y_column],
            mode='markers',
            name='Detected Peaks',
            marker=dict(size=8, symbol='circle', color='blue')
        ))

        # Plot detected peaks
        fig.add_trace(go.Scatter(
            x=best_fit_peaks[x_column],
            y=best_fit_peaks[y_column],
            mode='markers',
            name='Detected Peaks',
            marker=dict(size=8, symbol='circle', color='red')
        ))

        # Plot detected peaks
        fig.add_trace(go.Scatter(
            x=spectra_valleys[x_column],
            y=spectra_valleys[y_column],
            mode='markers',
            name='Detected Valleys',
            marker=dict(size=8, symbol='circle', color='cyan')
        ))

        # Plot detected peaks
        fig.add_trace(go.Scatter(
            x=best_fit_valleys[x_column],
            y=best_fit_valleys[y_column],
            mode='markers',
            name='Detected Valleys',
            marker=dict(size=8, symbol='circle', color='pink')
        ))

        # Update layout for better interactivity
        fig.update_layout(
            height=600,
            width=900,
            title=f"Spectra and Best Fit Data Overlay for {patient}",
            xaxis_title="Wavelength",
            yaxis_title="Intensity",
            hovermode='x unified'
        )

        display(fig)

# Initial render
render_plot(patient_dropdown.value)

# Observer for updating plot when selection changes
patient_dropdown.observe(lambda change: render_plot(change['new']), names='value')